In [1]:
import numpy as np
import pickle as pk
import sys
import os
import optparse
import logging

def parse():
    optparser = optparse.OptionParser()
    optparser.add_option("-d", "--data", dest="train", default="data/hansards", help="Data filename prefix (default=data)")
    optparser.add_option("-e", "--english", dest="english", default="e", help="Suffix of English filename (default=e)")
    optparser.add_option("-f", "--french", dest="french", default="f", help="Suffix of French filename (default=f)")
    optparser.add_option("--eps", dest="eps", default=1e-3, type="float", help="Default error bound for stopping the EM algorithm (default=1e-3)")
    optparser.add_option("-n", "--num_sentences", dest="num_sents", default=100000000000, type="int", help="Number of sentences to use for training and alignment")
    optparser.add_option("-l", "--lower", action="store_true", dest="lower", help="Convert corpus to lower case (defualt=False)")
    optparser.add_option("-g", "--debugging", action="store_true", dest="debugging", help="Turn on debugging mode (defualt=False)")
    optparser.add_option("-s", "--save", dest="save_path", default="ibm1.pkl", help="Model save path")
    optparser.add_option("-r", "--save_r", dest="save_path_r", default="ibm1_r.pkl", help="Reversed model save path")
    optparser.add_option("-p", "--pretrained", dest="load_path", default="", help="Model save path")
    optparser.add_option("-t", "--pretrained_r", dest="load_path_r", default="", help="Reversed model save path")
    optparser.add_option("-m", "--max_iterations", dest="max_iters", default=500, type="int", help="Max number of iterations to train")
    optparser.add_option("-c", "--continue", dest="iter_now", type=int, default=None, help="Continue with iteration number provided by this option. Need to have -p option turned on")
    (opts, _) = optparser.parse_args()

    f = f"{opts.train}.{opts.french}"
    e = f"{opts.train}.{opts.english}"

    test(opts)
    return opts, f, e


def test (a) -> None:
    if a.iter_now is not None:
        assert a.load_path
        
    if not a.load_path:
        c, d = os.path.splitext(a.save_path)
        c = f"{c}_num{a.num_sents}_eps{a.eps}"
        e = c + d
        f, g = os.path.split(e)
        f = os.path.join(f, c)
        assert not os.path.exists(f), f"{f} occupied."
        os.makedirs(f)
        a.save_path = os.path.join(f, g)
        
    if not a.load_path_r:
        c, d = os.path.splitext(a.save_path_r)
        c = f"{c}_num{a.num_sents}_eps{a.eps}"
        e = c + d
        f, g = os.path.split(e)
        f = os.path.join(f, c)
        assert not os.path.exists(f), f"{f} occupied."
        os.makedirs(f)
        a.save_path = os.path.join(f, g)
        
    


class Tokenizer:
    def __init__(self, corpus, case_sensitive=True) -> None:
        words_set = set()
        for inpu in corpus:
            for choice in inpu:
                if case_sensitive:
                    words_set.add(choice)
                else:
                    choice = choice.lower()
                    words_set.add(choice)
        self.toWord = list(words_set)
        self.toIndex = {choice:index for index, choice in enumerate(self.toWord)}
        self.V = len(self.toWord)
    
    def encode(self, inpu):

        encoderesult = [self.toIndex[word] for word in inpu]

        return encoderesult
    
    def decode(self, inpuIndex):

        decoderesult = [self.toWord[id] for id in inpuIndex]   
        return decoderesult
    




class IBMModel1:
    def __init__(self, bilingual_text, source_tokenizer, target_tokenizer, save_directory, tolerance=1e-5, max_iterations=50):
        tokenized_text = [[source_tokenizer.encode(pair[0]), target_tokenizer.encode(pair[1])] for pair in bilingual_text]
        self.bilingual_text = tokenized_text
        self.source_tokenizer = source_tokenizer
        self.target_tokenizer = target_tokenizer
        self.source_vocab_size = source_tokenizer.V
        self.target_vocab_size = target_tokenizer.V
        self.translation_prob = np.ones((self.source_vocab_size, self.target_vocab_size)) / self.target_vocab_size
        self.save_directory = save_directory

        self.tolerance = tolerance
        self.max_iterations = max_iterations
        self.has_converged = False
        self.change = -1
    
    def save(self, iteration_count):
        base_name, file_extension = os.path.splitext(o.save_path)
        base_name = f"{base_name}_iter{iteration_count}"
        path = base_name + file_extension
        pk.dump(self, open(path, "wb"))

    def train(self, current_iteration=0):
        iteration = current_iteration
        while (iteration < self.max_iterations) and (not self.has_converged):
            iteration += 1
            logging.info(f"Iteration {iteration}, change={self.change:.8f}")
            logging.debug(f"translation_prob(the|la)={self.get_prob('the', 'la')}")
            logging.debug(f"translation_prob(of|la)={self.get_prob('of', 'la')}")

            count_matrix = np.zeros((self.source_vocab_size, self.target_vocab_size))
            total_vector = np.zeros(self.source_vocab_size)

            for source_sentence, target_sentence in self.bilingual_text:
                sentence_total = {}
                for target_word in target_sentence:
                    sentence_total[target_word] = 0
                    for source_word in source_sentence:
                        sentence_total[target_word] += self.translation_prob[source_word, target_word]

                for target_word in target_sentence:
                    for source_word in source_sentence:
                        count_matrix[source_word, target_word] += self.translation_prob[source_word, target_word]/sentence_total[target_word]
                        total_vector[source_word] += self.translation_prob[source_word, target_word]/sentence_total[target_word]
            
            self.change = 0
            for source_word_idx in range(self.source_vocab_size):
                for target_word_idx in range(self.target_vocab_size):
                    updated_prob = count_matrix[source_word_idx, target_word_idx] / total_vector[source_word_idx]
                    element_change = np.abs(updated_prob - self.translation_prob[source_word_idx, target_word_idx])
                    self.change = max(self.change, element_change)
                    self.translation_prob[source_word_idx, target_word_idx] = updated_prob

            if self.change < self.tolerance:
                self.has_converged = True
            self.save(iteration)

        if self.has_converged:
            logging.info(f"Model converged after {iteration} iteration under error {self.tolerance}!")
        else:
            logging.info(f"Training stopped after reaching max number of iterations {self.max_iterations}")

    def get_prob(self, target_word, source_word):
        try:
            target_index = self.target_tokenizer.toIndex[target_word]
            source_index = self.source_tokenizer.toIndex[source_word]
        except KeyError:
            return -1
        return self.translation_prob[source_index, target_index]

    
    

def align(model, model_r, bilingual_text):
    
    for (source_sentence, target_sentence) in bilingual_text:
        alignment = []
        alignment_r = []
        for (source_index, source_word) in enumerate(source_sentence): 
            highest_probability = 0
            best_target_index = 0
            for (target_index, target_word) in enumerate(target_sentence):
                probability = model.get_prob(target_word, source_word)
                if probability > highest_probability:
                    best_target_index = target_index
                    highest_probability = probability
            if(highest_probability > 0):
                alignment.append([source_index,best_target_index])

        for (target_index, target_word) in enumerate(target_sentence):
            highest_probability = 0
            best_index = 0
            for (source_index, source_word) in enumerate(source_sentence):
                probability = model_r.get_prob(source_word, target_word)
                if probability >highest_probability:
                    best_index = source_index
                    highest_probability = probability
            if(highest_probability > 0):
                alignment_r.append([best_index,target_index])
    
        for i in range(len(source_sentence)):
            for j in range(len(target_sentence)):
                if [i, j] in alignment and [i, j] in alignment_r:
                    sys.stdout.write("%i-%i " % (i, j))

        sys.stdout.write("\n")

        
        
        
if __name__ == "__main__": 
    o, f, e = parse()
    loglvl = logging.DEBUG if o.debugging else logging.INFO
    logging.basicConfig(level=loglvl)
    b = [[s.strip().split() for s in p] for p in zip(open(f), open(e))][:o.num_sents]
    br = [[s.strip().split() for s in p] for p in zip(open(e), open(f))][:o.num_sents]
    if o.lower:
        b = [[[word.lower() for word in fst],[word.lower() for word in est]] for fst, est in b]
    ft = [sent[0] for sent in b]
    et = [sent[1] for sent in b]
    f_tokenizer = Tokenizer(ft)
    e_tokenizer = Tokenizer(et)
    
    ftr = [sent[0] for sent in br]
    etr = [sent[1] for sent in br]
    f_tokenizer_r = Tokenizer(et)
    e_tokenizer_r = Tokenizer(ft)

  

    logging.info(f"french:{f_tokenizer.V}, english:{e_tokenizer.V}")
    if o.load_path:
        ibm = pk.load(open(o.load_path, "rb"))
        if o.iter_now is not None:
            ibm.train(o.iter_now)
    else:
        ibm = IBMModel1(b, f_tokenizer, e_tokenizer, o.save_path, tolerance=o.eps, max_iterations=5)
        ibm.train()
        
    if o.load_path_r:
        ibm_r = pk.load(open(o.load_path_r, "rb"))
        if o.iter_now is not None:
            ibm_r.train(o.iter_now)
    else:
        ibm_r = IBMModel1(br, f_tokenizer_r, e_tokenizer_r, o.save_path_r, tolerance=o.eps, max_iterations=5)
        ibm_r.train()

    align(ibm, ibm_r, b)

OSError: [Errno 22] Invalid argument: 'data/hansards.C:\\Users\\charl\\AppData\\Roaming\\jupyter\\runtime\\kernel-ffeb52ad-4d08-443c-9473-1a32d7efb05a.json'